# Machine Learning Project #1
by Georgios Karakostas

## Agenda

One challenge that hosts looking to rent their living space face is determining the optimal nightly rent price. In many areas, renters are presented with a good selection of listings and can filter on criteria like price, number of bedrooms, room type and more. Since AirBnB is a marketplace, the amount a host can charge on a nightly basis is closely linked to the dynamics of the marketplace. 

As a host, if we try to charge above market price for a living space we'd like to rent, then renters will select more affordable alternatives which are similar to ours. On the other hand, ff we set our nightly rent price too low, we'll miss out on potential revenue.

One strategy we could use is to:

* Find a few listings that are similar to ours,
* average the listed price for the ones most similar to ours,
* set our listing price to this calculated average price.

The process of discovering patterns in existing data to make a prediction is called **machine learning**. In our case, we want to use data on local listings to predict the optimal price for us to set. In this project, we'll explore a specific machine learning technique called **k-nearest neighbors**, which mirrors the strategy we just described. Before we dive further into machine learning and k-nearest neighbors, let's get familiar with the dataset we'll be working with.

While AirBnB doesn't release any data on the listings in their marketplace, a separate group named [Inside AirBnN](https://stackoverflow.com/questions/25692293/inserting-a-link-to-a-webpage-in-an-ipython-notebook) has extracted data on a sample of the listings for many of the major cities on the website. In this project, we'll be working with their dataset from October 3, 2015 on the listings from Washington, D.C., the capital of the United States. Each row in the dataset is a specific listing that's available for renting on AirBnB in the Washington, D.C. area.

To make the dataset less cumbersome to work with, we've removed many of the columns in the original dataset and renamed the file to `dc_airbnb.csv`. Here are the columns we kept:

* `host_response_rate`: the response rate of the host
* `host_acceptance_rate`: number of requests to the host that convert to rentals
* `host_listings_count`: number of other listings the host has
* `latitude`: latitude dimension of the geographic coordinates
* `longitude`: longitude part of the coordinates
* `city`: the city the living space resides
* `zipcode`: the zip code the living space resides
* `state`: the state the living space resides
* `accommodates`: the number of guests the rental can accommodate
* `room_type`: the type of living space (Private room, Shared room or Entire home/apt
* `bedrooms`: number of bedrooms included in the rental
* `bathrooms`: number of bathrooms included in the rental
* `beds`: number of beds included in the rental
* `price`: nightly price for the rental
* `cleaning_fee`: additional fee used for cleaning the living space after the guest leaves
* `security_deposit`: refundable security deposit, in case of damages
* `minimum_nights`: minimum number of nights a guest can stay for the rental
* `maximum_nights`: maximum number of nights a guest can stay for the rental
* `number_of_reviews`: number of reviews that previous guests have left

## First Observation
Let's read the dataset into Pandas and become more familiar with it.

In [1]:
import pandas as pd
dc_listings = pd.read_csv('dc_airbnb.csv')
dc_listings.iloc[0]

host_response_rate                  92%
host_acceptance_rate                91%
host_listings_count                  26
accommodates                          4
room_type               Entire home/apt
bedrooms                              1
bathrooms                             1
beds                                  2
price                           $160.00
cleaning_fee                    $115.00
security_deposit                $100.00
minimum_nights                        1
maximum_nights                     1125
number_of_reviews                     0
latitude                          38.89
longitude                      -77.0028
city                         Washington
zipcode                           20003
state                                DC
Name: 0, dtype: object

# Part 2: K-nearest Neighbors
Here's the strategy we want to use:

* Find a few similar listings.
* Calculate the average nightly rental price of these listings.
* Set the average price as the price for our listing.

The k-nearest neighbors algorithm is similar to this strategy.

There are 2 things we need to unpack in more detail:

* The similarity metric.
* How to choose the k value.

In this project, we'll define what similarity metric we're going to use. Then, we'll implement the **k-nearest neighbors** algorithm and use it to suggest a price for a new, unpriced listing. We'll use a **k value** of 5 for now. Later on, we'll evaluate how good the suggested prices are, choose the optimal **k value**, and more. To that end.

### Euclidean Distance

The similarity metric works by comparing a fixed set of numerical features, another word for attributes, between 2 observations, or living spaces in our case. When trying to predict a continuous value, like price, the main similarity metric that's used is **Euclidean distance**. Here's the general formula for Euclidean distance:

$$d = \sqrt{(q_1 - p_1)^2 + (q_2 - p_2)^2 + \ldots + (q_n - p_n)^2}$$

where $q_1$ to $q_n$ represent the feature values for one observation and $p_1$ to $p_n$ represent the feature values for the other observation.

To begin with, we'll use just one feature to familiarize ourselves with the machine learning workflow. Since we're only using one feature, this is known as the **univariate case**. Here's how the formula looks like for the univariate case:

$$d = \sqrt{(q_1 - p_1)^2}$$

The square root and the squared power cancel and the formula simplifies to:

$$d = |q_1 - p_1|$$

Let's suppose that the living space  we want to rent can accommodate 3 people. We'll first calculate the Euclidean distance, using just the `accommodates` feature, between the first living space in the dataset and "our own". To that end, we proceed:

In [2]:
import numpy as np
our_acc_value = 3
first_living_space_value = dc_listings.iloc[0]['accommodates']
first_distance = np.abs(first_living_space_value - our_acc_value)
first_distance

1

The Euclidean distance between the first row in the `dc_listings` Dataframe and our own living space is 1. How do we know if this is high or low? If you look at the Euclidean distance equation itself, the lowest value you can achieve is 0. This happens when the value for the feature is exactly the same for both observations we're comparing. If $p_1 = q_1$, then $|q_1 - p_1|$  which results in $d = 0$. The closer to 0 the distance the more similar the living spaces are.

Next, we'll calculate the Euclidean distance between each value in the `accommodates` column from `dc_listings` and the value `3`, which is the number of people our listing accommodates. 

In [3]:
new_listing = 3
dc_listings['distance'] = dc_listings['accommodates'].apply(lambda x: np.abs(x - new_listing))
dc_listings['distance'].value_counts()

1     2294
2      503
0      461
3      279
5       73
4       35
7       22
6       17
9       12
13       8
8        7
12       6
11       4
10       2
Name: distance, dtype: int64

It looks like there are quite a few, 461 to be precise, living spaces that can accommodate 3 people just like ours. This means the 5 "nearest neighbors" we select after sorting all will have a distance value of 0. If we sort by the `distance` column and then just select the first 5 living spaces, we would be biasing the result to the ordering of the dataset.

Let's instead randomize the ordering of the dataset and then sort the Dataframe by the `distance` column. This way, all of the living spaces with the same number of bedrooms will still be at the top of the Dataframe but will be in random order across the first 461 rows.

In [4]:
np.random.seed(1)
dc_listings = dc_listings.loc[np.random.permutation(len(dc_listings))]
dc_listings = dc_listings.sort_values('distance')
print(dc_listings.iloc[0:10]['price'])

577     $185.00
2166    $180.00
3631    $175.00
71      $128.00
1011    $115.00
380     $219.00
943     $125.00
3107    $250.00
1499     $94.00
625     $150.00
Name: price, dtype: object


## Part 3: Average price

Before we can select the 5 most similar living spaces and compute the average price, we need to clean the `price` column. Right now, the `price` column contains comma characters (`,`) and dollar sign characters and is formatted as a text column instead of a numeric one. We need to remove these values and convert the entire column to the `float` datatype. Then, we can calculate the average price. To that end, we proceed: 

In [5]:
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')
mean_price = dc_listings.iloc[0:5]['price'].mean()
mean_price

156.6

We've just made our first prediction! Based on the average price of other listings that accommdate 3 people, we should charge **156.6** dollars per night for a guest to stay at our living space. In the next sections, we'll dive into evaluating how good of a prediction this is.

To conclude this section, we'll write a more general function that can suggest the optimal price for other values of the `accommodates` column. The `dc_listings` Dataframe has information specific to our living space, e.g. the `distance` column. To save time, we'll reset the `dc_listings` Dataframe to a clean slate and only keep the data cleaning and randomization we did since those weren't unique to the prediction we were making for our living space.

In [6]:
# Brought along the changes we made to the `dc_listings` Dataframe.
dc_listings = pd.read_csv('dc_airbnb.csv')
stripped_commas = dc_listings['price'].str.replace(',', '')
stripped_dollars = stripped_commas.str.replace('$', '')
dc_listings['price'] = stripped_dollars.astype('float')
dc_listings = dc_listings.loc[np.random.permutation(len(dc_listings))]

def predict_price(new_listing):
    temp_df = dc_listings.copy()
    temp_df['distance'] = temp_df['accommodates'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbors = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbors.mean()
    return(predicted_price)

acc_one = predict_price(1)
acc_two = predict_price(2)
acc_four = predict_price(4)

We now have a function that can predict the price for any living space we want to list as long as we know the number of people it can accommodate. The function we wrote represents a **machine learning model**, which means that it outputs a prediction based on the input to the model.

   ## Testing quality of predictions

In [7]:
train_df = dc_listings.iloc[0:2792]
test_df = dc_listings.iloc[2792:]

def predict_price(new_listing):
    temp_df = train_df.copy()
    temp_df['distance'] = temp_df['accommodates'].apply(lambda x: np.abs(x - new_listing))
    temp_df = temp_df.sort_values('distance')
    nearest_neighbor_prices = temp_df.iloc[0:5]['price']
    predicted_price = nearest_neighbor_prices.mean()
    return(predicted_price)

test_df['predicted_price'] = test_df['accommodates'].apply(predict_price)

/Users/georgioskarakostas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [8]:
test_df['error'] = np.absolute(test_df['predicted_price'] - test_df['price'])
mae = test_df['error'].mean()
print(mae)

59.22599355531667


/Users/georgioskarakostas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
